# SIADS 521, Assignment 4

By: Grant Jason

## Rule et al's computational narrative rules used

### 1. Tell a story
Through the notebook, I attempt to use both markdown cells and comments inside code cells to help the viewer understand what I am doing and the story I am trying to tell with the data

### 2. Document the process, not the results
Similar to above, I try to ensure the notebook is very linear and clearly commented throughout

### 3. Use cell divisions to make steps clear
I try to clearly break up each major step in the workflow into its own cell

### 4. Modularize code
I contain each chart into its own function so when I need to generate a dashoboard using them later, I just call the function instead of re-writing the code

### 6. Use version control
This notebook was built using a private git repository on my github profile: https://github.com/grantajason

In [13]:
# import statements

import pandas as pd                                 # Data manipulation
import numpy as np                                  # For any stats or math needed
import matplotlib.pyplot as plt                     # General plotting and formatting
import seaborn as sns                               # More advanced charting
import ipywidgets as widgets                        # For interaction
from matplotlib import style                        # For setting overall style
from IPython.display import display, clear_output   # For dashboarding
from ipywidgets import interact, interactive        # For interaction

## Data Preparation
To start, lets bring in the inital data set and perform some general cleansings that will likely be applicable to all charts. Any chart specific cleansing or transformations I will do within the function that is used to create each chart.

In [ ]:
# bring in strava data

strava = pd.read_csv('~/Downloads/strava.csv')

# convert timestamp to datetime object for easier manipulation
strava['timestamp'] = pd.to_datetime(strava['timestamp'])
strava['month_name'] = strava['timestamp'].dt.month_name()

# calculate actual degrees in case I use lat/lon
strava['lat_deg'] = strava['position_lat'] * (180/2**31)
strava['lon_deg'] = strava['position_long'] * (180/2**31)


,Air Power,Cadence,Form Power,Ground Time,Leg Spring Stiffness,Power,Vertical Oscillation,altitude,cadence,datafile,...,position_lat,position_long,speed,timestamp,unknown_87,unknown_88,unknown_90,month_name,lat_deg,lon_deg
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,activities/2675855419.fit.gz,...,NaN,NaN,0.0,2019-07-08 21:04:03,0.0,300.0,NaN,July,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,activities/2675855419.fit.gz,...,NaN,NaN,0.0,2019-07-08 21:04:04,0.0,300.0,NaN,July,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,activities/2675855419.fit.gz,...,NaN,NaN,1316.0,2019-07-08 21:04:07,0.0,300.0,NaN,July,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3747.0,77.0,activities/2675855419.fit.gz,...,504432050.0,-999063637.0,1866.0,2019-07-08 21:04:14,0.0,100.0,NaN,July,42.281006,-83.740547
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3798.0,77.0,activities/2675855419.fit.gz,...,504432492.0,-999064534.0,1894.0,2019-07-08 21:04:15,0.0,100.0,NaN,July,42.281043,-83.740622


## Chart #1
### How much did Prof Brooks' cardio health increase over the summer?
The first thing I want to do to understand how much Prof Brooks' cardio health improved over the summer. To do this, I want to look at the distribution of heart rates over the time periods available in the data set. I decided to use a violin plot to show this distribution, with the X Axis being the month and the Y axis being the heart rate distribution.

In [ ]:
def cardio_health_violin(ax):
    # create copy of initial data set to avoid errors later
    cardio = strava.copy()

    sns.set_theme(style=style.library['fivethirtyeight'])

    violin = sns.violinplot(x=strava['month_name'], y=strava['heart_rate'], ax=ax)

    ax.set_title('Heart Rate Distribution by Month', pad=20)
    ax.set_xlabel('Month', labelpad=20)
    ax.set_ylabel('Heart Rate in BPM', labelpad=20)

cardio_health_violin()

TypeError: cardio_health_violin() missing 1 required positional argument: 'ax'